In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision
from torchvision.io import read_image
import torchvision.transforms as transforms
from tqdm.notebook import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using:', device)

Using: cuda


In [2]:
TRAIN_FOLDER_PATH = "C:/Users/rjrao/Downloads/train"
TEST_FOLDER_PATH = "C:/Users/rjrao/Downloads/test"

In [3]:
class PartiallyModifiedResNet18(nn.Module):
    def __init__(self, n_classes, remove_layers=2, frozen_layers=6):
        super(PartiallyModifiedResNet18, self).__init__()

        # Load pre-trained ResNet-18 model
        resnet18 = torchvision.models.resnet18(pretrained=True)
        # Remove the specified number of layers from the end
        resnet_layers = list(resnet18.children())[:-remove_layers]
        #  last_resnet_layers = list(resnet18.children())[-2:]
        self.features = nn.Sequential(*resnet_layers)

        # Freeze the specified number of layers from the beginning
        for i, param in enumerate(self.features.parameters()):
            param.requires_grad = i < frozen_layers

        # Modify the last fully connected layer
        #  self.adaptive_poll = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.relu = nn.ReLU()
        self.avgpool = nn.AvgPool2d(kernel_size=(7,7), stride=(1,1))
        self.fc = nn.Linear(in_features=1*1*512, out_features=1000)
        self.out = nn.Linear(in_features=1000, out_features=n_classes)
        #  self.fc = nn.Linear(resnet18.fc.in_features, n_classes)

    def forward(self, x):

        x = self.features(x)
        #  x = self.adaptive_poll(x)

        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.relu(self.fc(x))
        x = self.out(x)
        #  x = x.view(x.size(0), -1)
        #  x = self.fc(x)
        return x

In [4]:
# dataset 
train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])
test_dataset = torchvision.datasets.ImageFolder(root = TEST_FOLDER_PATH, transform = train_transforms)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

train_dataset = torchvision.datasets.ImageFolder(root = TRAIN_FOLDER_PATH, transform = train_transforms)
n_classes = len(set(train_dataset.targets))
train, val = torch.utils.data.random_split(train_dataset, [4000, 5232-4000])
train_loader = DataLoader(dataset=train, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val, batch_size=64, shuffle=True)

In [5]:
# CONFIGURING MODEL PARAMETES

n_batches = len(train_loader)

model = PartiallyModifiedResNet18(n_classes=n_classes,remove_layers=2,frozen_layers=6).to(device)
weight=torch.Tensor([0.74,0.26]).to(device)
criterion = nn.CrossEntropyLoss(weight=weight)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
print(model)

C:\Users\rjrao\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\rjrao\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


PartiallyModifiedResNet18(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [6]:
# TRAINING:
epochs = 10

for epoch in range(epochs):
    loss_list = []
    num_correct, num_samples = 0, 0
    for batch_num, (images, labels) in tqdm(enumerate(train_loader), total = n_batches):
        images = images.to(device)
        labels = labels.to(device)
        
        predictions = model(images)
        loss = criterion(predictions, labels)
        loss_list.append(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _, train_predictions = predictions.max(1)
        num_correct += (train_predictions==labels).sum()
        num_samples += predictions.size(0)
    
    training_accuracy = float(num_correct)/float(num_samples)*100 
    
    val_loss_list = []
    val_num_correct, val_num_samples = 0, 0
    model.eval()
    with torch.no_grad():
        for val_batch_num, (val_images, val_labels) in enumerate(val_loader):
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)

            val_predictions = model(val_images)
            _, val_preds = val_predictions.max(1)
            
            val_num_correct += (val_preds==val_labels).sum()
            val_num_samples += val_preds.size(0)
            
            val_loss_list.append(criterion(val_predictions, val_labels))
        
    val_accuracy = float(val_num_correct)/float(val_num_samples)*100
    model.train()

    print(f'Epoch: {epoch + 1} | Accuracy: {training_accuracy:.2f} | loss: {sum(loss_list)/len(loss_list)}', end='')
    print(f' | Validation Accuracy: {val_accuracy:.2f} | Validation loss: {sum(val_loss_list)/len(val_loss_list)}')

  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 1 | Accuracy: 89.55 | loss: 0.28239789605140686 | Validation Accuracy: 93.02 | Validation loss: 0.29707181453704834


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 2 | Accuracy: 94.92 | loss: 0.1353379786014557 | Validation Accuracy: 95.62 | Validation loss: 0.16465608775615692


  0%|          | 0/63 [00:00<?, ?it/s]

...

Epoch: 8 | Accuracy: 97.95 | loss: 0.054633188992738724 | Validation Accuracy: 96.92 | Validation loss: 0.1342964619398117


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 9 | Accuracy: 98.42 | loss: 0.044216230511665344 | Validation Accuracy: 96.75 | Validation loss: 0.14372961223125458


  0%|          | 0/63 [00:00<?, ?it/s]

Epoch: 10 | Accuracy: 98.38 | loss: 0.04427209496498108 | Validation Accuracy: 97.56 | Validation loss: 0.11427439749240875


In [7]:
# Assuming you have a test DataLoader similar to train and val DataLoader
test_num_correct, test_num_samples = 0, 0
test_loss_list = []

model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    for test_batch_num, (test_images, test_labels) in enumerate(test_loader):
        test_images = test_images.to(device)
        test_labels = test_labels.to(device)

        test_predictions = model(test_images)
        _, test_preds = test_predictions.max(1)

        test_num_correct += (test_preds == test_labels).sum().item()
        test_num_samples += test_preds.size(0)

        test_loss_list.append(criterion(test_predictions, test_labels))

test_accuracy = float(test_num_correct) / float(test_num_samples) * 100
average_test_loss = sum(test_loss_list) / len(test_loss_list)

print(f'Test Accuracy: {test_accuracy:.2f} | Test Loss: {average_test_loss}')

Test Accuracy: 89.58 | Test Loss: 0.31505775451660156
